# hierarchical algorithm to identify semi-rigid domains an super-domains in proteins
Written by Mahtab.

This script is the implementation of hierarchical algorithm to identify semi-rigid domains an super-domains in proteins.

In [87]:
# Nessecary imports
import numpy as np
from scipy.spatial.distance import cdist
import MDAnalysis as mda
from MDAnalysis.analysis import align, rms
from typing import *
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm
from collections import defaultdict
from sklearn.mixture import GaussianMixture
from scipy.stats import norm
import statistics

In [88]:
# Global variables
topology_file = '/path/to/topology/file' 
trajectory_file = '/path/to/trajectory/file'     
begin = 0   # the starting frame (The first frame of the trajectory we want to include in clustering).

In [89]:
# Define a Union-Find (or Disjoin-Set) class
class UnionFind:
    def __init__(self):
        self.parent = {}
    
    # Function that finds the root of a node.
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        
        return self.parent[x]
    
    # Function that joins the groups of x and y
    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x != root_y:
            self.parent[root_x] = root_y

    # Function that adds x to the data structure if it wasn't before.
    def add(self, x):
        if x not in self.parent:
            self.parent[x] = x
        

In [90]:
# Functions.

# Computes the S matrix (normalized standard deviation of distance variation) from the topology and trajectory files.
def S_generator(topology_file: str, trajectory_file: str, selection: str = "protein and name CA") -> np.ndarray:

    # Load the topology and trajectory files in to the universe "u"
    u = mda.Universe(topology_file, trajectory_file)

    # Get the coordinates of the atoms.
    atoms = u.select_atoms(selection)
    n_atoms = atoms.n_atoms
    n_frames = len(u.trajectory[begin:])

    print('\nnumber of selected atoms: ', n_atoms)
    print('\nnumber of frames: ', n_frames)

    # arrays for storing sum of pair distances and sum of squared pair distances
    sum_pairDistances = np.zeros((n_atoms, n_atoms))
    sum_squaredDistances = np.zeros((n_atoms, n_atoms))

    # compute pairwise distances over each frame.
    for f, ts in enumerate(u.trajectory[begin:]):
       print(f"\n frame: {f}")
       # compute pairwise distances for frame f
       distances = cdist(atoms.positions, atoms.positions, metric='euclidean') 

       # Accumulate the pairwise distances and squeared pairwise distances
       sum_pairDistances += distances
       sum_squaredDistances += distances ** 2
    
    # compute average of pairwise distance over the trajectory
    average_pairDistances = sum_pairDistances / n_frames

    # compute the average of squared pairwise distances over the trajectory
    average_squaredDistances = sum_squaredDistances / n_frames

    # compute variance (squared distance variations)
    variance = average_squaredDistances - average_pairDistances ** 2

    # Caluculate standard deviation (STDDV) matrix shape(n_atoms * n_atoms).
    S = np.sqrt(variance)
    
    print('\nS matrix for constructing domains is generated!')
    # print(f'\n S: {S}')

    return S



# Function that computes the contiguous domains.
# Starts from the first CA and compares the STDV of CA1 and CAi with 
# T (threshold). if STDV CA1&CAi < T, they're from the same domain.
# The first domain finishes when STDV CA1&CAi >= T. Continues this 
# until we reach the last CA.
def construct_domains(S:np.ndarray, T: float):

    n_atoms = S.shape[0]
    domains = {}    # dictionary to store the domains
    # start and end of the current domain
    start = 0
    end = 0
    domain_index = 1

    # continue creating domains until we have reached the last CA
    while start <= (n_atoms-1):
        try:
            # find the first index where S[start][?] >= T
            end = (np.where(S[start][start:] >= T))[0][0]
        except IndexError:
            # If such an index doesn't exist, it means that the domains ends with the last CA.
            end = n_atoms - start
        
        # add the domain to the domain dictionary
        domains[domain_index] = {}
        domains[domain_index]["start"] = start + 1
        domains[domain_index]["end"] = start + end
        domains[domain_index]["length"] = end
        
        # set the start to the starting point of next domain
        start += end 
        domain_index +=1
    
    return domains



# Array that computes the coordinates of the selected atoms of a trajectory.
# Coordinate matrix generator function
# Reads the position of atoms from the topology and trajectory files and generates and returns coordinate matrix
# "selection" is an optional parameter which is by default "protein and name CA" for selecting backbone C-alphas. It is the string corresponding to the group of atoms we want to consider in clustering
def coordinate_generator(topology_file: str, trajectory_file: str, selection: str = "protein and name CA") -> np.ndarray:

    # Load the topology and trajectory files in to the universe "u"
    u = mda.Universe(topology_file, trajectory_file)

    # Get the coordinates of the atoms.
    atoms = u.select_atoms(selection)
    n_atoms = atoms.n_atoms
    n_frames = len(u.trajectory[begin:])
    coordinates = np.empty((n_frames, n_atoms, 3))

    print('\nnumber of selected atoms: ', n_atoms)
    print('\nnumber of frames: ', n_frames)

    # Align the current frame to the ref frame
    align.alignto(u, u, select=selection, weights="mass")

    # fill out the coordinates array
    for i, ts in enumerate(u.trajectory[begin:]):
        coordinates[i] = atoms.positions.copy()

    print('\nCoordinates are generated!')

    return coordinates



# Function to calculate the average over the trajectory of minimum distance between any two domains.
# It chooses the minimum distance among the
# pairwise distances between the atoms of the two domains for each domains, 
# and then takes the average of those values over the trajectory.
# Note: domain_ids are assmued to be base 1.
def avrg_min_dist_calculator(domains_id1: int, domains_id2: int, domains: dict, coordinates: np.ndarray):

    n_frames = coordinates.shape[0]

    # get the start and end CA of each of the domains
    d1_start = domains[domains_id1]['start'] - 1    # turn to base 0
    d1_end = domains[domains_id1]['end']
    
    d2_start = domains[domains_id2]['start'] - 1    # turn to base 0
    d2_end = domains[domains_id2]['end']

    mindists_per_frame = np.empty(n_frames)
    for frame in range(n_frames):
        # extract the relevant coordinates for each domain in the specified frame
        d1_coords = coordinates[frame, d1_start:d1_end, :]
        d2_coords = coordinates[frame, d2_start:d2_end, :]

        # compute minimum pairwise distance between the two atoms of the domains in current frame
        minDistance = np.min(cdist(d1_coords, d2_coords, metric='euclidean'))

        # Add the obtained minimum distance to the mindists_per_frame array
        mindists_per_frame[frame] = minDistance

    # Take the average of mindists_per_frame to obtain the average min distance between the two domains
    avrg_mindist = np.mean(mindists_per_frame)

    return avrg_mindist


# Function to construct the average minimum distance matrix between any two domains
def build_distance_matrix(domains: dict, coordinates: np.ndarray):
    n_domains = len(domains)

    # construct an array of average minimum distances bewteen any two domains
    avrg_mindists = np.empty((n_domains, n_domains))

    for i in range(n_domains):
        for j in range(n_domains):
            avrg_mindists[i, j] = avrg_min_dist_calculator(i+1, j+1, domains, coordinates)

    return avrg_mindists


# Function to calculate the average standard deviation between two given domains.
# domains_ids should be base 1.
def avrg_stdv_calculator(domains_id1: int, domains_id2: int, domains: dict, coordinates: np.ndarray):
    n_frames = coordinates.shape[0]
    n_domains = len(domains)

    # get the start and end CA of each of the domains
    d1_start = domains[domains_id1]['start'] - 1    # turn to base 0
    d1_end = domains[domains_id1]['end']
    d1_size = d1_end - d1_start

    d2_start = domains[domains_id2]['start'] - 1    # turn to base 0
    d2_end = domains[domains_id2]['end']
    d2_size = d2_end - d2_start

    # extract the relevant coordinates for each domain 
    d1_coords = coordinates[:, d1_start:d1_end, :]
    d2_coords = coordinates[:, d2_start:d2_end, :]

    # arrays for storing sum of pair distances and sum of squared pair distances
    sum_pairDistances = np.zeros((d1_size, d2_size))
    sum_squaredDistances = np.zeros((d1_size, d2_size))

    # compute pairwise distances between the CAs of the two domains over each frame
    for frame in range(n_frames):
        # compute pairwise distances for this frame
        distances = cdist(d1_coords[frame], d2_coords[frame], metric='euclidean')

        # Accumulate the pairwise distance and the squared pairwise distance
        sum_pairDistances += distances
        sum_squaredDistances += distances ** 2

    # compute average of pairwise distances over the trajectory
    average_pairDistances = sum_pairDistances / n_frames
    
    # compute the average of squared pairwise distances over the trajectory
    average_squaredDistances = sum_squaredDistances / n_frames

    # compute variance (squared distance variations)
    variance = average_squaredDistances - average_pairDistances ** 2

    # Caluculate standard deviation (STDDV) matrix shape(n_atoms * n_atoms).
    S_all = np.sqrt(variance)

    # Calculate the average of the standard deviations of distances between the two domains
    S_average = np.mean(S_all)

    return S_average

# computes the standard deviation of distance between any two domains.
def S_domains(domains:dict, coordinates: np.ndarray):
    n_domains = len(domains)

    # construct an array of average standard deviation between any two domains 
    avrg_stdv = np.empty((n_domains, n_domains))

    for d1 in range(1, n_domains+1):
        for d2 in range(1, n_domains+1):
            if d1 == d2:
                avrg_stdv[d1-1, d2-1] = float(0)
            else:
                avrg_stdv[d1-1, d2-1] = avrg_stdv_calculator(d1, d2, domains, coordinates)

    return avrg_stdv



def solo_domains(domains: dict, superdomains: dict):
    all_domains = set(range(1, len(domains)+1))
    superd_domains = set()
    for key, value in superdomains.items():
        superd_domains.update(value['domains'])

    return all_domains - superd_domains



# Function that constructs superdomain through a graph problem approach:
# Applies the thresholds, and then finds the disjoint sets in the filtered domains.
# The domains that are not part of a superdomains in the end will be grouped into their own superdomain.
def hierarchical_superdomains(domains: dict, coordinates: np.ndarray, t_dist: float, t_stddv: float):
    n_domains = len(domains)
    # compute the min distance and standard deviation of minimum distance between any two domains.
    D_matrix = build_distance_matrix(domains, coordinates)  # D_matrix = average minimum distance matrix
    S_matrix = S_domains(domains, coordinates)
    print(f'\nS_matrix: {S_matrix}')
    print(f'\nD_matrix: {D_matrix}')

    # Ensure that D_matrix and S_matrix are both of shape (n_domains, n_domains)
    assert D_matrix.shape == (n_domains, n_domains) and S_matrix.shape == (n_domains, n_domains), f'Matrices D and S should be of shape {(n_domains, n_domains)}'

    # initiate the union find class
    uf = UnionFind()

    # Boolean mask for the thresholds
    mask = (D_matrix < t_dist) & (S_matrix < t_stddv)
    print(f'\nmask: {mask}')

    # Iterate over the matrix indices and union the pairs that satisfy the thresholds 
    test = []
    for i in range(n_domains):
        for j in range(i+1, n_domains):  # ensure that i != j and avoid counting twice (since D and S are symmetric matrices)
            if mask[i, j]:
                # Add i and j to the Union Find structure and union them
                uf.add(i+1)
                uf.add(j+1)
                uf.union(i+1, j+1)
                test.append((i+1, j+1))
    
    print(f'\npairs: {test}')
    
    # Group the elements by their root (construct superdomains by merging domains)
    disjoint_sets = defaultdict(list)
    for ele in uf.parent:
        root = uf.find(ele)
        disjoint_sets[root].append(ele)
    
    print(f'\ndisjoint_sets: {disjoint_sets}')

    # construct the superdomains dictionary based on the disjoin sets
    superdomains = {}
    superdomain_key = 1

    for group in disjoint_sets.values():
        superdomains[superdomain_key] = {
            'domains': group,
            'number of domains': len(group),
            'CAs': [(domains[i]['start'], domains[i]['end']) for i in group]
        }
        superdomain_key += 1

    # put all the domains that did not get merged into their own superdomain
    solos = solo_domains(domains, superdomains)
    for domain_id in solos:
        superdomains[superdomain_key] = {
            "domains": [domain_id],
            "number of domains": 1,
            "CAs": [(domains[domain_id]['start'], domains[domain_id]['end'])] 
        }
        superdomain_key += 1

    print("\nSuperdomain construction is over!")
    print(f"\nFinal superdomains: {superdomains}")

    return superdomains


In [ ]:
# Generate S matrix
S = S_generator(topology_file, trajectory_file)
n_CAs = S.shape[0]

In [ ]:
# Plot everything in one plot with subplots 
# Generate a grid for the plots
upper_S = S[np.triu_indices(n_CAs, k=1)].reshape(-1, 1)

# Matplotlib preambles
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

# Font sizes
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 12

plt.subplots_adjust(hspace=0.1, wspace=0.1)  # Experiment with these values

rows, cols = 3, 4
n_components = range(1, rows*cols + 1)  # Adjust to fit within the grid
AIC_scores_ds1 = []
# shared axis grid
fig, axes = plt.subplots(rows, cols, figsize=(12,10), sharex=True, sharey=True)
# get a palette from matplotlib
palette = [cm.tab10(i) for i in range(10) if i != 7]#cm.tab10


for i, ax in enumerate(axes.flat):
    if i < len(n_components):
        n = n_components[i]
        gmm = GaussianMixture(n_components=n, random_state=0)
        gmm.fit(upper_S)
        AIC_scores_ds1.append(gmm.aic(upper_S))

        # Plot histogram of data on the corresponding subplot
        ax.hist(upper_S, bins=100, density=True, alpha=0.8, color='#d3d3d3') 

        # Overlay each Gaussian component from GMM
        x_range = np.linspace(min(upper_S), max(upper_S), 1000)
        log_prob = gmm.score_samples(x_range.reshape(-1, 1))
        pdf = np.exp(log_prob)
        ax.plot(x_range, pdf, ':k', linewidth=2.5, label='Overall PDF')

        # Plot each individual Gaussian
        for j, (mean, covar, weight) in enumerate(zip(gmm.means_, gmm.covariances_, gmm.weights_)):
            component_pdf = weight * (1 / np.sqrt(2 * np.pi * covar)) * np.exp(-(x_range - mean) ** 2 / (2 * covar))
            ax.plot(x_range, component_pdf.ravel(), lw=2.0, color=palette[j % len(palette)])
        
        ax.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
        # aDD SUBPLOT TITLE
        ax.set_title(F'n={n}', weight='bold')
      
    else:
        fig.delaces(ax)


# Set shared labels and gridlines
for ax in axes[:, 0]: # First column
    ax.set_ylabel('Probabiliy Density', fontsize=14, weight='bold')
for ax in axes[-1, :]: # last row
    ax.set_xlabel('Distance Variation (Å)', fontsize=14, weight='bold')    

# Adjust layout and save
plt.tight_layout() # leaves no space since there are no legends
plt.show()

In [ ]:
# Plot the AIC scores for different n_components in a separate figure
# Matplotlib preambles
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

plt.figure(figsize=(10, 6))
plt.plot(n_components, AIC_scores_ds1, marker='o', markersize=8, linestyle='-', linewidth=2, markeredgecolor='black', color='tab:green')
plt.grid(color='gray', linestyle='--', linewidth=0.4, alpha=0.6)
plt.title('AIC Scores for Gaussian Mixture Models', fontsize=16, weight='bold', pad=15)
plt.xlabel('Number of Components', fontsize=14, weight='bold')
plt.ylabel('AIC Scores', fontsize=14, weight='bold')

plt.xlim(0)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

print(f'\nAIC scores for ds: {AIC_scores_ds1}')


In [ ]:
# optimal_n_components for ds chosen based on the plots
optimal_n_components = 4

# fit GMM with the optimal_n_components
gmm_optimal = GaussianMixture(n_components=optimal_n_components, random_state=0)
gmm_optimal.fit(upper_S)

# set the t_domain (threshold) as the average of the mean of the first two Gaussians
mean1 = np.sort(gmm_optimal.means_, axis=0)[0,0]
mean2 = np.sort(gmm_optimal.means_, axis=0)[1,0]
print(f'mean1: {mean1}, mean2: {mean2}')
ds = np.mean([mean1, mean2])
print(f'Determined threshold ds: {ds}')
print(np.sort(gmm_optimal.means_, axis=0))

In [ ]:
# Construct the domains
print(f"ds: {ds}")
domains = construct_domains(S, ds)
n_domains = len(domains)
print(f'\ndomains: {domains}')

In [ ]:
# Construct the matrices required for superdomain construction
x = coordinate_generator(topology_file, trajectory_file)
n_frames, n_CAs, _ = x.shape
S2 = S_domains(domains, x)
mindists = build_distance_matrix(domains, x)

In [ ]:
# Plot everything in one plot wiht subplots 
# Generate a grid for the plots
upper_S2 = S2[np.triu_indices(n_domains, k=1)].reshape(-1, 1)

# Matplotlib preambles
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

# Font sizes
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 12

plt.subplots_adjust(hspace=0.1, wspace=0.1)  # Experiment with these values

rows, cols = 3, 3
n_components = range(1, rows*cols + 1)  # Adjust to fit within the grid
AIC_scores_t_stdvv1 = []
# shared axis grid
fig, axes = plt.subplots(rows, cols, figsize=(9, 9), sharex=True, sharey=True)
# get a palette from matplotlib
palette = [cm.tab10(i) for i in range(10) if i != 7]#cm.tab10


for i, ax in enumerate(axes.flat):
    if i < len(n_components):
        n = n_components[i]
        gmm = GaussianMixture(n_components=n, random_state=0)
        gmm.fit(upper_S2)
        AIC_scores_t_stdvv1.append(gmm.aic(upper_S2))

        # Plot histogram of data on the corresponding subplot
        ax.hist(upper_S2, bins=15, density=True, alpha=0.8, color='#d3d3d3') 

        # Overlay each Gaussian component from GMM
        x_range = np.linspace(min(upper_S2), max(upper_S2), 1000)
        log_prob = gmm.score_samples(x_range.reshape(-1, 1))
        pdf = np.exp(log_prob)
        ax.plot(x_range, pdf, ':k', linewidth=2.5, label='Overall PDF')

        # Plot each individual Gaussian
        for j, (mean, covar, weight) in enumerate(zip(gmm.means_, gmm.covariances_, gmm.weights_)):
            component_pdf = weight * (1 / np.sqrt(2 * np.pi * covar)) * np.exp(-(x_range - mean) ** 2 / (2 * covar))
            ax.plot(x_range, component_pdf.ravel(), lw=2.0, color=palette[j % len(palette)])
        
        ax.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
        # aDD SUBPLOT TITLE
        ax.set_title(F'n={n}', weight='bold')
        ax.set_ylim(0, 1.2)

    else:
        fig.delaces(ax)


# Set shared labels and gridlines
for ax in axes[:, 0]: # First column
    ax.set_ylabel('Probabiliy Density', fontsize=14, weight='bold')
for ax in axes[-1, :]: # last row
    ax.set_xlabel('Average Distance Variation (Å)', fontsize=14, weight='bold')    

# Adjust layout and save
plt.tight_layout() # leaves no space since there are no legends
plt.show()

In [ ]:
# Plot the AIC scores for different n_components in a separate figure
# Matplotlib preambles
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

plt.figure(figsize=(10, 6))
plt.plot(n_components, AIC_scores_t_stdvv1, marker='o', markersize=8, linestyle='-', linewidth=2, markeredgecolor='black', color='tab:green')
plt.grid(color='gray', linestyle='--', linewidth=0.4, alpha=0.6)
plt.title('AIC Scores for Gaussian Mixture Models', fontsize=16, weight='bold', pad=15)
plt.xlabel('Number of Components', fontsize=14, weight='bold')
plt.ylabel('AIC Scores', fontsize=14, weight='bold')

plt.xlim(0)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

print(f'\nAIC scores for t_superdomain: {AIC_scores_t_stdvv1}')


In [ ]:
# optimal_n_components for t_stddv (t_superdomains) chosen based on the plots
optimal_n_components = 6

# fit GMM with the optimal_n_components
gmm_optimal = GaussianMixture(n_components=optimal_n_components, random_state=0)
gmm_optimal.fit(upper_S2)

# set the t_domain (threshold) as the average of the mean of the first two Gaussians
mean1 = np.sort(gmm_optimal.means_, axis=0)[0,0]
mean2 = np.sort(gmm_optimal.means_, axis=0)[1,0]
print(f'mean1: {mean1}, mean2: {mean2}')
t_stddv = np.mean([mean1, mean2])
print(f'Determined threshold t_stddv: {t_stddv}')
print(np.sort(gmm_optimal.means_, axis=0))

In [ ]:
# Plot everything in one plot wiht subplots 
# Generate a grid for the plots
upper_mindists = mindists[np.triu_indices(n_domains, k=1)].reshape(-1, 1)

# Matplotlib preambles
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

# Font sizes
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 12

plt.subplots_adjust(hspace=0.1, wspace=0.1)  # Experiment with these values

rows, cols = 2, 3
n_components = range(1, rows*cols + 1)  # Adjust to fit within the grid
AIC_scores_t_dist1 = []
# shared axis grid
fig, axes = plt.subplots(rows, cols, figsize=(9, 6), sharex=True, sharey=True)
# get a palette from matplotlib
palette = [cm.tab10(i) for i in range(10) if i != 7]#cm.tab10


for i, ax in enumerate(axes.flat):
    if i < len(n_components):
        n = n_components[i]
        gmm = GaussianMixture(n_components=n, random_state=0)
        gmm.fit(upper_mindists)
        AIC_scores_t_dist1.append(gmm.aic(upper_mindists))

        # Plot histogram of data on the corresponding subplot
        ax.hist(upper_mindists, bins=10, density=True, alpha=0.8, color='#d3d3d3') 

        # Overlay each Gaussian component from GMM
        x_range = np.linspace(min(upper_mindists), max(upper_mindists), 1000)
        log_prob = gmm.score_samples(x_range.reshape(-1, 1))
        pdf = np.exp(log_prob)
        ax.plot(x_range, pdf, ':k', linewidth=2.5, label='Overall PDF')

        # Plot each individual Gaussian
        for j, (mean, covar, weight) in enumerate(zip(gmm.means_, gmm.covariances_, gmm.weights_)):
            component_pdf = weight * (1 / np.sqrt(2 * np.pi * covar)) * np.exp(-(x_range - mean) ** 2 / (2 * covar))
            ax.plot(x_range, component_pdf.ravel(), lw=2.0, color=palette[j % len(palette)])
        
        ax.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)
        # aDD SUBPLOT TITLE
        ax.set_title(F'n={n}', weight='bold')
        ax.set_ylim(0, 0.10)
        
    else:
        fig.delaces(ax)


# Set shared labels and gridlines
for ax in axes[:, 0]: # First column
    ax.set_ylabel('Probabiliy Density', fontsize=14, weight='bold')
for ax in axes[-1, :]: # last row
    ax.set_xlabel('Average Minimum Distance (Å)', fontsize=14, weight='bold')    

# Adjust layout and save
plt.tight_layout() # leaves no space since there are no legends
plt.show()

In [ ]:
# Plot the AIC scores for different n_components in a separate figure
# Matplotlib preambles
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

plt.figure(figsize=(10, 6))
plt.plot(n_components, AIC_scores_t_dist1, marker='o', markersize=8, linestyle='-', linewidth=2, markeredgecolor='black', color='tab:green')
plt.grid(color='gray', linestyle='--', linewidth=0.4, alpha=0.6)
plt.title('AIC Scores for Gaussian Mixture Models', fontsize=16, weight='bold', pad=15)
plt.xlabel('Number of Components', fontsize=14, weight='bold')
plt.ylabel('AIC Scores', fontsize=14, weight='bold')

plt.xlim(0)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

print(f'\nAIC scores for t_dist: {AIC_scores_t_dist1}')


In [ ]:
# optimal_n_components for t_dist (d_superdomains) chosen based on the plots
optimal_n_components = 8

# fit GMM with the optimal_n_components
gmm_optimal = GaussianMixture(n_components=optimal_n_components, random_state=0)
gmm_optimal.fit(upper_mindists)

# set the t_domain (threshold) as the average of the mean of the first two Gaussians
mean1 = np.sort(gmm_optimal.means_, axis=0)[0,0]
mean2 = np.sort(gmm_optimal.means_, axis=0)[1,0]
print(f'mean1: {mean1}, mean2: {mean2}')
t_dist = mean1.copy() #np.mean([mean1, mean2])
print(f'Determined threshold t_dist: {t_dist}')
print(np.sort(gmm_optimal.means_, axis=0))

In [ ]:
# Construct superdomains
superds = hierarchical_superdomains(domains, x,t_dist, t_stddv)

In [ ]:
# generate domain assignment matrix from "domains" dictionary
n_domains = len(domains)
domain_assignment = np.zeros((n_CAs, n_domains), dtype=int)

for domain_idx, domain_info in domains.items():
    first = domain_info['start'] - 1    # convert to index starting from 0 (0 based indexing)
    last = domain_info['end']   # already is on 0 based index (ranges are exclusive in the line below)
    domain_index = domain_idx - 1   # convert to 0 based indexing
    domain_assignment[first:last, domain_index] = 1

print(domain_assignment)

In [ ]:
# generate super domain assignment matrix from superdomains.
n_superdomains = len(superds)
_, n_CA, _ = x.shape
superd_assignment = np.zeros((n_CA, n_superdomains), dtype=int)


for superd_idx, superd_info in superds.items():
    for pair in superd_info['CAs']:
        first = pair[0] - 1 # convert to 0 based index
        last = pair[1]
        superdomain_index = superd_idx - 1  # convert to 0 based index
        superd_assignment[first:last, superdomain_index] = 1

print(superd_assignment)

In [161]:
# Function to add superdomain ID feature to atoms of trajectory file and make a pdb file.
# Sets the superdomain ID of all atoms in a residue to the superdomain ID of CA of that residue.
def id_adder(topology_file: 'str', trajectory_file: str, superd_assignment: np.ndarray, output_file: 'str'):
    # Load the topology and trajectory files into the uiverse
    u = mda.Universe(topology_file, trajectory_file)
    # Create new attributes for the atoms
    u.add_TopologyAttr('tempfactors')
    protein = u.select_atoms('protein')
    
    # Extract the superdomain IDs from superdomain_assignment matrix
    superd_ids = np.argmax(superd_assignment, axis=1) 
    # Add one to the superdomain numbers so that they start with 1 and end in 10 (for simplicity, so that we wouldn't have 0s).
    superd_ids += 1
    

    for residue, ID in zip(protein.residues, superd_ids):
        residue.atoms.tempfactors = ID
    
    u.atoms.write(output_file)  # If you want to write only protein: protein.write..., If you want the whole system: u.atoms.write...

    print(f"Trajectory including cluster IDs written to {output_file}.")

In [ ]:
# make a pdb file for domains
id_adder(topology_file, trajectory_file, domain_assignment, "\path\to\store\domains\output\file")

In [ ]:
# make a pdb file for super domains
id_adder(topology_file, trajectory_file, superd_assignment, "\path\to\store\domains\output\file")

In [113]:
# Cell to define functions that validate the results of he super-domain construction (for thesis)
# The goal is to prove that the stddv inside super-domains is less than outside (i.e. the super-domains are indeed rigid)

# Function to calculate average stddv aong two coordinate sets
def helper_stddv(x1: np.ndarray, x2: np.ndarray):
    n_frames = x1.shape[0]
    size1 = x1.shape[1]
    size2 = x2.shape[1]

    # arrays for storing sum of pair distances and sum of squared pair distances
    sum_pairDistances = np.zeros((size1, size2))
    sum_squaredDistances = np.zeros((size1, size2))

    # compute pairwise distances between the CAs of the two domains over each frame
    for frame in range(n_frames):
        # compute pairwise distances for this frame
        distances = cdist(x1[frame], x2[frame], metric='euclidean')

        # Accumulate the pairwise distance and the squared pairwise distance
        sum_pairDistances += distances
        sum_squaredDistances += distances ** 2

    # compute average of pairwise distances over the trajectory
    average_pairDistances = sum_pairDistances / n_frames
    
    # compute the average of squared pairwise distances over the trajectory
    average_squaredDistances = sum_squaredDistances / n_frames

    # compute variance (squared distance variations)
    variance = average_squaredDistances - average_pairDistances ** 2

    # Caluculate standard deviation (STDDV) matrix shape(n_atoms * n_atoms).
    S_all = np.sqrt(variance)

    # Calculate the average of the standard deviations of distances between the two domains
    S_average = np.mean(S_all)

    return S_average    

# Function that calculates the average stddv among CAs inside a super-domain
# super domain ids should be base 1.
def inner_stddv(superdomains: dict, superd_id: int, coordinates: np.ndarray):
    # extract the CA coordinates of the given super-domain
    n_frames = coordinates.shape[0]
    ca_indices = superdomains[superd_id]['CAs']
    assert len(ca_indices) > 0
    superd_coords = np.zeros((n_frames, 0, 3))

    for start, end in ca_indices:
        superd_coords = np.concatenate((superd_coords, coordinates[:, start-1:end, :]), axis=1)
    
    # compute the average stddv inside the super-domain
    result = helper_stddv(superd_coords, superd_coords)

    return result


# Function to calculate the stddv among two super-domains
def outer_stddv(superdomains: dict, id1: int, id2: int, coordinates: np.ndarray):
    # extract the CA coordinates of the given super-domains
    n_frames = coordinates.shape[0]
    ca_indices1 = superdomains[id1]['CAs']
    ca_indices2 = superdomains[id2]['CAs']
    assert len(ca_indices1) > 0
    assert len(ca_indices2) > 0
    superd1_coords = np.empty((n_frames, 0, 3))
    superd2_coords = np.empty((n_frames, 0, 3))

    for start, end in ca_indices1:
        superd1_coords = np.concatenate((superd1_coords, coordinates[:, start-1:end, :]), axis=1)

    for start, end in ca_indices2:
        superd2_coords = np.concatenate((superd2_coords, coordinates[:, start-1:end, :]), axis=1)

    # calculate the average stddv among the two superdomains
    result = helper_stddv(superd1_coords, superd2_coords)

    return result

In [ ]:
# Cell to test the functions that validate the results of he super-domain construction (for thesis)
# The goal is to prove that the stddv inside super-domains is less than outside (i.e. the super-domains are indeed rigid)
for i in range(1, n_superdomains+1):
    inner = inner_stddv(superds, i, x)
    print(f"\ninner stddv superd {i}: {inner:.3f}")
    outers = [
        outer_stddv(superds, i, j, x)
        for j in range(1, n_superdomains+1) if j != i
    ]

    avg_outer = np.mean(outers)
    min_outer = np.min(outers)
    max_outer = np.max(outers)
    
    # print(f"\n{outers}")
    print(f"\nminimum outer stddv: {min_outer:.3f}")
    print(f"\naverage outer stddv: {avg_outer:.3f}")
    print(f"\nmaximum outer stddv: {max_outer:.3f}")

    if inner < avg_outer and inner < min_outer:
        print(" Validation: PASS")
    else:
        print(" Validation: FAIL")
